In [1]:
import pandas as pd
import numpy as np

In [2]:
TRAIN_RATIO = 0.85
VALIDATION_RATIO = 0.15

In [2]:
df = pd.read_csv('../data/interim/credit_card_transactions-ibm_v2.csv')
df.head(3)

,User,Card,Year,Month,Day,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,Hour,Minute,Timestamp,Date,Day of Week
0,0,0,2002,9,1,134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,0,6,21,2002-09-01 06:21:00,2002-09-01,Вс
1,0,0,2002,9,1,38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,0,6,42,2002-09-01 06:42:00,2002-09-01,Вс
2,0,0,2002,9,2,120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,0,6,22,2002-09-02 06:22:00,2002-09-02,Пн


In [3]:
df.set_index('Timestamp', inplace=True)
df.index.names=['Timestamp']
df = df.sort_index() # sort index
df.head(3)

,User,Card,Year,Month,Day,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,Hour,Minute,Date,Day of Week
Timestamp,,,,,,,,,,,,,,,,,,
1991-01-02 07:10:00,791,1,1991,1,2,68.00,Swipe Transaction,2027553650310142703,Burke,VA,22015.0,5541,NaN,0,7,10,1991-01-02,Ср
1991-01-02 07:17:00,791,1,1991,1,2,-68.00,Swipe Transaction,2027553650310142703,Burke,VA,22015.0,5541,NaN,0,7,17,1991-01-02,Ср
1991-01-02 07:21:00,791,1,1991,1,2,113.62,Swipe Transaction,2027553650310142703,Burke,VA,22015.0,5541,NaN,0,7,21,1991-01-02,Ср


In [4]:
df.tail(3)

,User,Card,Year,Month,Day,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,Hour,Minute,Date,Day of Week
Timestamp,,,,,,,,,,,,,,,,,,
2020-02-28 23:56:00,1366,2,2020,2,28,132.73,Chip Transaction,-7398558035733466800,Rockville Centre,NY,11570.0,5812,NaN,0,23,56,2020-02-28,Пт
2020-02-28 23:56:00,1300,0,2020,2,28,51.29,Online Transaction,-6458444334611773637,ONLINE,NaN,NaN,4784,NaN,0,23,56,2020-02-28,Пт
2020-02-28 23:58:00,446,3,2020,2,28,42.29,Online Transaction,-7421093378627544099,ONLINE,NaN,NaN,5311,NaN,0,23,58,2020-02-28,Пт


In [5]:
df_train = df[(df['Year'] >= 2015) & (df['Year'] <= 2017)]
df_val = df[df['Year'] == 2018]
df_test = df[df['Year'] == 2019]

In [6]:
df = None

In [7]:
df_train.shape

(5133655, 18)

In [8]:
df_val.shape

(1721615, 18)

In [9]:
df_test.shape

(1723938, 18)

In [10]:
df_train.describe()

,User,Card,Year,Month,Day,Amount,Merchant Name,Zip,MCC,Is Fraud?,Hour,Minute
count,5.133655e+06,5.133655e+06,5.133655e+06,5.133655e+06,5.133655e+06,5.133655e+06,5.133655e+06,4.457627e+06,5.133655e+06,5.133655e+06,5.133655e+06,5.133655e+06
mean,1.006234e+03,1.290352e+00,2.016004e+03,6.534492e+00,1.572567e+01,4.296813e+01,-4.809246e+17,5.102985e+04,5.558804e+03,1.385952e-03,1.238641e+01,2.959286e+01
std,5.700813e+02,1.370590e+00,8.167591e-01,3.446103e+00,8.796324e+00,8.112104e+01,4.766753e+18,2.939080e+04,8.807619e+02,3.720257e-02,5.075254e+00,1.731781e+01
min,0.000000e+00,0.000000e+00,2.015000e+03,1.000000e+00,1.000000e+00,-5.000000e+02,-9.222899e+18,1.001000e+03,1.711000e+03,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.170000e+02,0.000000e+00,2.015000e+03,4.000000e+00,8.000000e+00,9.060000e+00,-4.530601e+18,2.831200e+04,5.300000e+03,0.000000e+00,9.000000e+00,1.500000e+01
50%,1.011000e+03,1.000000e+00,2.016000e+03,7.000000e+00,1.600000e+01,2.925000e+01,-7.948098e+17,4.733800e+04,5.499000e+03,0.000000e+00,1.200000e+01,3.000000e+01
75%,1.483000e+03,2.000000e+00,2.017000e+03,1.000000e+01,2.300000e+01,6.307000e+01,3.189517e+18,7.754500e+04,5.812000e+03,0.000000e+00,1.600000e+01,4.500000e+01
max,1.999000e+03,8.000000e+00,2.017000e+03,1.200000e+01,3.100000e+01,5.155360e+03,9.222877e+18,9.992800e+04,9.402000e+03,1.000000e+00,2.300000e+01,5.900000e+01


In [11]:
df_val.describe()

,User,Card,Year,Month,Day,Amount,Merchant Name,Zip,MCC,Is Fraud?,Hour,Minute
count,1.721615e+06,1.721615e+06,1721615.0,1.721615e+06,1.721615e+06,1.721615e+06,1.721615e+06,1.494430e+06,1.721615e+06,1.721615e+06,1.721615e+06,1.721615e+06
mean,1.005353e+03,1.275023e+00,2018.0,6.528249e+00,1.573014e+01,4.287903e+01,-4.972069e+17,5.086893e+04,5.557165e+03,1.446897e-03,1.239466e+01,2.958236e+01
std,5.707656e+02,1.362633e+00,0.0,3.446947e+00,8.798331e+00,8.055068e+01,4.765355e+18,2.937833e+04,8.807919e+02,3.801059e-02,5.076575e+00,1.731910e+01
min,0.000000e+00,0.000000e+00,2018.0,1.000000e+00,1.000000e+00,-5.000000e+02,-9.222899e+18,1.001000e+03,1.711000e+03,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.170000e+02,0.000000e+00,2018.0,4.000000e+00,8.000000e+00,9.040000e+00,-4.589688e+18,2.821500e+04,5.300000e+03,0.000000e+00,9.000000e+00,1.500000e+01
50%,1.010000e+03,1.000000e+00,2018.0,7.000000e+00,1.600000e+01,2.918000e+01,-8.685521e+17,4.711200e+04,5.499000e+03,0.000000e+00,1.200000e+01,3.000000e+01
75%,1.482000e+03,2.000000e+00,2018.0,1.000000e+01,2.300000e+01,6.300000e+01,3.189517e+18,7.749400e+04,5.812000e+03,0.000000e+00,1.600000e+01,4.500000e+01
max,1.999000e+03,8.000000e+00,2018.0,1.200000e+01,3.100000e+01,5.682220e+03,9.222877e+18,9.990100e+04,9.402000e+03,1.000000e+00,2.300000e+01,5.900000e+01


In [12]:
df_test.describe()

,User,Card,Year,Month,Day,Amount,Merchant Name,Zip,MCC,Is Fraud?,Hour,Minute
count,1.723938e+06,1.723938e+06,1723938.0,1.723938e+06,1.723938e+06,1.723938e+06,1.723938e+06,1.495120e+06,1.723938e+06,1.723938e+06,1.723938e+06,1.723938e+06
mean,1.004711e+03,1.276298e+00,2019.0,6.525915e+00,1.571612e+01,4.274106e+01,-4.955698e+17,5.103297e+04,5.559918e+03,1.210600e-03,1.238867e+01,2.956494e+01
std,5.702081e+02,1.367541e+00,0.0,3.446948e+00,8.795041e+00,8.012487e+01,4.765159e+18,2.944270e+04,8.817533e+02,3.477263e-02,5.077021e+00,1.732823e+01
min,0.000000e+00,0.000000e+00,2019.0,1.000000e+00,1.000000e+00,-5.000000e+02,-9.222899e+18,1.001000e+03,1.711000e+03,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.170000e+02,0.000000e+00,2019.0,4.000000e+00,8.000000e+00,9.010000e+00,-4.536641e+18,2.830300e+04,5.300000e+03,0.000000e+00,9.000000e+00,1.500000e+01
50%,1.009000e+03,1.000000e+00,2019.0,7.000000e+00,1.600000e+01,2.921000e+01,-8.981400e+17,4.736200e+04,5.499000e+03,0.000000e+00,1.200000e+01,3.000000e+01
75%,1.482000e+03,2.000000e+00,2019.0,1.000000e+01,2.300000e+01,6.298000e+01,3.189517e+18,7.757300e+04,5.812000e+03,0.000000e+00,1.600000e+01,4.500000e+01
max,1.999000e+03,8.000000e+00,2019.0,1.200000e+01,3.100000e+01,6.613440e+03,9.222877e+18,9.982900e+04,9.402000e+03,1.000000e+00,2.300000e+01,5.900000e+01


In [13]:
df_train['Is Fraud?'].mean(), df_val['Is Fraud?'].mean(), df_test['Is Fraud?'].mean()

(0.0013859521140396072, 0.0014468972447382255, 0.0012106003812202063)

In [14]:
df_train.to_csv('../data/processed/train.csv',
              header=df_train.columns,
              index=False)

In [15]:
df_val.to_csv('../data/processed/val.csv',
              header=df_val.columns,
              index=False)

In [16]:
df_test.to_csv('../data/processed/test.csv',
              header=df_test.columns,
              index=False)

In [18]:
df_train = None
df_val = None
df_test = None